## Solver

In [ ]:
import numpy as np

# input: 9x9 numpy array, empty cell = 0
# output: 9x9 numpy array: if not solution all array entries should be -1
# backtracking depth-first search with constraint propagation


# Returns True if sudoku argument is invalid 
def initial_invalid(sudoku):

    # Check row
    for i in range(9):
        dup_lst = []
        for j in range(9):
            if sudoku[j][i] != 0:
                if sudoku[j][i] in dup_lst:
                    return True

                else:
                    dup_lst.append(sudoku[j][i])

    # Check column
    for i in range(9):
        dup_lst = []
        for j in range(9):
            if sudoku[i][j] != 0:
                if sudoku[i][j] in dup_lst:
                    return True

                else:
                    dup_lst.append(sudoku[i][j])
    
    # Check boxes
    for box_x in range(3):
        for box_y in range(3):
            dup_lst = []
            # y-axis/columns
            for i in range(box_y*3, box_y*3 + 3):
                # x-axis/rows
                for j in range(box_x * 3, box_x*3 + 3):
                    if sudoku[i][j] in dup_lst:
                        return True
                    

# Returns (row, col) of an empty cell with the least number of possible values.
# False if no empty cell exists.
def get_cell_least_val(sudoku, possible_values):
    min_possible_values = 10
    min_row = -1
    min_col = -1
    
    for i in range(len(sudoku)):
        for j in range(len(sudoku[0])):
            if sudoku[i][j] == 0:
                num_possible_numbers = sum(possible_values[i][j])
                
                if num_possible_numbers < min_possible_values:
                    min_possible_values = num_possible_numbers
                    min_row = i
                    min_col = j
                    
    if min_possible_values < 10:
        return (min_row, min_col)
    else:
        return False


    
# Remove invalid values from possible_values in the given row, col and box containing cell[row][col]
# Returns list of tuple values that have been udpated
def remove(possible_values, row, col, value):
    
    cells_changed = []
    for c_index, c in enumerate(possible_values[row]):
        if c[value - 1] == 1:
            cells_changed.append((row, c_index))
            c[value - 1] = 0

    for r in range(9):
        if possible_values[r][col][value - 1] == 1:
            cells_changed.append((r, col))
            possible_values[r][col][value - 1] = 0

    # Check box
    box_x = col // 3
    box_y = row // 3

    # y-axis/columns
    for i in range(box_y*3, box_y*3 + 3):
        # x-axis/rows
        for j in range(box_x * 3, box_x*3 + 3):
            if possible_values[i][j][value - 1] == 1:
                cells_changed.append((i,j))
                possible_values[i][j][value - 1] = 0
    return cells_changed


# Loop possible_values and reset values for each (row, col) tuple in the changed cells list
def reset(possible_values, value, changed_cells):
    for row, col in changed_cells:
        possible_values[row][col][value - 1] = 1

        

# Returns True if board solved 
# Returns False if board insolvable - all guesses exhausted 
def solver(sudoku, possible_values):
    next_cell = get_cell_least_val(sudoku, possible_values)
    if not next_cell:
        return True
    else:
        row, col = next_cell

    for guessed_value in range(1, 10):
        if possible_values[row][col][guessed_value - 1] != 1:
            continue
        # update possible values and sudoku board for recursive call
        changed_cells = remove(possible_values, row, col, guessed_value)
        sudoku[row][col] = guessed_value

        if solver(sudoku, possible_values):
            return True
        # reset possible values and board for next iteration
        reset(possible_values, guessed_value, changed_cells)
        sudoku[row][col] = 0

    return False


def sudoku_solver(sudoku):

    if initial_invalid(sudoku):
        return np.full((9, 9), -1)
    
    #each row represents a list of possible values for a position in the 9x9 sudoku board
    possible_values = np.ones((9,9,9), dtype=int)
    
    
    # initialize possible values based on already existing values
    for row in range(9):
        for col in range(9):
            value = sudoku[row][col]
            if value != 0:
                remove(possible_values, row, col, value)

    if solver(sudoku, possible_values):
        return sudoku
    else:
        return np.full((9, 9), -1)

## Tests

In [ ]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

In [ ]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break